In [2]:
import pandas as pd
import numpy as np

data_file_name = 'data.csv'
labels_file_name = 'what_answer_codes_mean.csv'
questions_file_name = 'what_questsions_are.csv'

data = pd.read_csv(data_file_name)
labels = pd.read_csv(labels_file_name)
questions = pd.read_csv(questions_file_name)

In [14]:
# remove data where country is Nan
data = data[data['Residency'] != 'Nan']
data = data.dropna()
data = data[data['var'] == data['var'].astype(str)]

# remove data in what_questions_are where "question" is not a string
questions = questions[questions['question'] == questions['question'].astype(str)]
# get and remove the question data where [var] is NA
group_header_statements = questions[questions['row_id'] == 0]
questions = questions[questions['row_id'] != 0]

# get unique countries
countries = data['Residency'].unique()
# get unique questions
unique_questions = data['var'].unique()

In [15]:
country_dict = {}

for index, row in data.iterrows():
    country_dict[row['Residency']] = country_dict.get(row['Residency'], {})
    country_dict[row['Residency']][row['var']] = country_dict[row['Residency']].get(row['var'], [])
    country_dict[row['Residency']][row['DemGen']] = country_dict[row['Residency']].get(row['DemGen'], [])
    country_dict[row['Residency']][row['quota_age']] = country_dict[row['Residency']].get(row['quota_age'], [])
    country_dict[row['Residency']][row['var']].append(row['code'])
    country_dict[row['Residency']][row['DemGen']].append(row['DemGen'])
    country_dict[row['Residency']][row['quota_age']].append(row['quota_age'])

In [ ]:
df = pd.DataFrame.from_dict(country_dict, orient='index')
print(df)
df.to_json('data.json')

In [23]:
df = pd.read_json('data.json')
# print(df)
# print column names
print(df['GenSocTrust'])

IT    [1, 7, 3, 4, 5, 3, 5, 1, 2, 3, 4, 5, 4, 2, 4, ...
UK    [5, 4, 6, 5, 4, 3, 5, 2, 5, 6, 5, 5, 1, 6, 5, ...
US    [6, 2, 4, 1, 3, 4, 2, 3, 5, 4, 5, 1, 5, 4, 4, ...
AU    [5, 3, 6, 6, 6, 1, 3, 6, 5, 1, 2, 5, 3, 3, 1, ...
DE    [2, 4, 4, 3, 5, 5, 2, 4, 3, 4, 2, 1, 7, 1, 3, ...
ES    [2, 2, 6, 5, 6, 1, 2, 5, 5, 1, 1, 4, 6, 2, 2, ...
MX    [2, 1, 1, 3, 1, 5, 4, 2, 4, 2, 3, 1, 5, 4, 5, ...
SE    [3, 4, 5, 5, 5, 7, 4, 6, 4, 4, 2, 5, 3, 3, 4, ...
JP    [2, 4, 5, 6, 5, 4, 2, 3, 1, 5, 5, 3, 1, 4, 6, ...
KR    [5, 7, 5, 7, 1, 3, 4, 4, 4, 5, 4, 5, 6, 3, 6, ...
CN    [6, 4, 5, 5, 7, 7, 7, 4, 6, 3, 6, 6, 4, 4, 3, ...
Name: GenSocTrust, dtype: object


In [ ]:
print(country_dict['IT'].keys())